<h1>Curso de Sistemas Inteligentes</h1>
<h2>Práctica de Algoritmos Genéticos</h2>
<h2>Periodo 2020-1</h2>

A continuación se propone una plantilla para la implementación de un algoritmo genético genérico. En esta práctica deberá completarse este código para resolver una tarea específica. Primero se definen las funciones auxiliares genéricas, luego las funciones que se deben reescribir para una tarea particular y al final está el programa principal.
Se supone que el individuo es una lista, su longitud depende del problema particular.

In [78]:
# This matrix will represent a representation for testing purposes of the task assignment problem.
# The values mTest[i][j] = k means the operation i from task n on the machine j requires k units of time.
# i refers to i % len(mTest[0]), n refers to int(i / len(mTest[0])), j refers to j + 1 for the purposes of this problem.
# The values given as INF indicates that given operation cannot be done on that machine.
# opOrder must be given as an input, indicating the tasks to which every operation belongs.
# The same way as M, T, OP, which will be the number of machines, tasks and operations.
from random import seed, randint, uniform
from math import isnan
from numpy.random import choice
INF = float('inf')
M, T, OP = 3, 3, 8
opOrder = [0, 0, 0, 1, 1, 2, 2, 2]
mTest = [[10, 8, INF], 
         [INF, 12, INF], 
         [4, 6, 5], 
         [11, 18, INF], 
         [20, INF, INF],
         [INF, 12, 16], 
         [7, 12, 4], 
         [14, 11, 9]]

In [79]:
# Función de selección
# Elige dos individuos aleatoriamente que van a ser cruzados
# generacion: lista de individuos que componen la generacion actual
# Retorna las posiciones de dos individuos en la generacion
def seleccion(generacion):
    tGen = len(generacion)
    ind1 = randint(0, tGen-1)
    ind2 = ind1
    while ind1 == ind2: ind2 = randint(0,tGen-1)
    return (generacion[ind1], generacion[ind2])

# gen = [[0, 0, 2, 2, 0, 0, 2, 2, 2], [2, 1, 0, 1, 1, 1, 1, 0, 0], [2, 0, 0, 1, 1, 1, 0, 0, 2], [1, 1, 0, 0, 1, 0, 0, 2, 0]]
# a,b = seleccion(gen)
# print(a)
# print(b)

In [80]:
# Roulette wheel selection or fitness selection. 
# First, I must evaluate the fitness of every chromosome within a generation. This will determine it's chance of being selected.
# Once that's done, I will generate a random number between 0 and the sum of all the fitness values.
# Now, I will start to add up the fitness values in a counter. 
# If that random number is greater than my counter, that means I haven't reached the selected individual.
# On the contrary, if it isn't, I will select that chromosome and set it's probability of being selected to 0.
def roulette(gen):
    fit, ind = [fitness(i) for i in gen], [0, 0]
    for j in range(len(ind)):
        ind1 = uniform(0, sum(fit))
        found = False
        c, i = 0, 0
        while(not found and i < len(gen)):
            if(ind1 <= c): found = True
            else:
                c = c + fit[i]
                i = i + 1
        ind[j] = i - 1
        fit[ind[j]] = 0
    return ([gen[i] for i in ind])
# gen = [[1, 1, 1, 0, 0, 2, 2, 2], [1, 1, 1, 1, 0, 2, 1, 1], [1, 1, 0, 1, 0, 2, 0, 1], [1, 1, 2, 0, 0, 1, 2, 0], [1, 1, 0, 1, 0, 1, 0, 0], [1, 1, 0, 1, 0, 1, 2, 0], [1, 1, 0, 2, 0, 1, 1, 2]]
# optRoulette(gen)

In [81]:
# Funcion de descarte de los individuos menos aptos
# generacion: lista de individuos que componen la generacion actual
# Retorna la generacion despues de eliminar la mitad menos apta
def descarte(generacion):
    tGen = len(generacion)
    return (generacion[:tGen//2])
# descarte(gen)

In [82]:
# Funcion de cruce
# Precondicion: ambos individuos tienen la misma longitud
# ind1 e ind2 son individuos de la generacion actual
#Retorna dos nuevos individuos obtenidos a partir de ind1 e ind2 por cruce
def cruce(ind1,ind2):
    tInd = len(ind1)
    pivot = randint(0,tInd-1)
    new1 = ind1[:pivot] + ind2[pivot:]
    new2 = ind2[:pivot] + ind1[pivot:]
    return (new1, new2)

#cruce([0, 0, 2, 0, 0, 2, 0, 0, 2], [0, 2, 1, 2, 0, 1, 1, 0, 2])

In [83]:
# Funcion de mutacion
# ind es un individuo de la generacion actual
# prob es un valor entre 0 y 1 que corresponde a la probabilidad de mutacion
# Retorna un individuo que puede ser identico al que entró o puede tener un cambio aleatorio en una posicion
def mutacion(ind, prob):
    p = randint(1,100)
    if p < prob*100: 
        tInd = len(ind)
        q = randint(0,tInd-1)
        ind[q] = randint(0, 2) # esto se debe ajustar de acuerdo a los valores adecuados para el individuo
    return (ind)

# mutacion([2, 2, 1, 2, 0, 0, 1, 1, 0],1.1)

In [84]:
# Funcion newInd
# Genera un nuevo individuo aleatorio
# Retorna el individuo construido
# Esta funcion debe ajustarse en terminos de la tarea que se vaya a resolver, pues la forma y longitud del individuo varian
# Chromosome will be represented as a list, e.g. chr_1 = [1, 2, 2, 2, 1, 1, 2, 1, 0]
# The value chr_1[i] = k means that the operation i of task j will be done in machine k.
# Given n * n = len(chr_1), i refers to i % n, j refers to int(i / n) and k refers to k + 1
# This is because the domain for the sets of this problem goes from 1 to N.
def newInd():
    ind = [randint(0, 2) for _ in range(OP)]
    return ind

# newInd()

In [85]:
# Funcion primeraGen
# nIndGen: numero de individuos por generacion
# Retorna la primera generacion poblada con el numero de individuos requeridos
# Esta funcion depende completamente del problema que se va a resolver, pues el individuo depende del problema
def primeraGen(nIndGen):
    generacion = []
    while len(generacion) < nIndGen: generacion.append(newInd())
    return generacion

# primeraGen(10)

In [ ]:
# Funcion fitness
# ind: es un individuo de la generacion actual
# Retorna un valor numerico que representa la aptitud del individuo
# Esta funcion depende completamente del problema que se va a resolver, pues el puntaje asociado al individuo depende del problema
# Minimizing the time C for the completion of an assignment of tasks means we must take the maximum time required to complete the tasks.
# That's why the fitness function will be the max amount of time.
# The arrays mTime and tTime represents the time for each machine and for each task, respectively.
# Since I need to check the previous value, I must initialize these two arrays with the time of the first operation.
# If the current operation is under the same task from the previous operation, I must check if that machine must wait before starting that operation.
# If it needs to wait, that wait time is added. After that, the time of that operation is added to the total time for that task.
# If the current operation is on a different task from the previous operation, I must add its time to the task time array.
# Once that's done, that operation's time will be added to the machine time.
# The score will be a normalized value done by dividing the maximum time from the three machines by the total time done by each machine.
# If this score is nan a.k.a. INF/INF, it will have an assigned score of 1.
# The lower the normalized value, the better, since it takes less time than the total amount. Because of this, the true score will be 1 - normalized value.

def fitness(ind):
    mTime, tTime = [0 for _ in range(M)], [0 for _ in range(T)]
    mTime[ind[0]], tTime[ind[0]] = mTest[0][ind[0]], mTest[0][ind[0]]
    for i in range(1, len(ind)):
        if(opOrder[i] == opOrder[i - 1]):
            mTime[ind[i]] = max(mTime[ind[i]], tTime[opOrder[i]]) + mTest[i][ind[i]]
            tTime[opOrder[i]] += mTest[i][ind[i]]
        else:
            tTime[opOrder[i]] += mTest[i][ind[i]] + mTime[ind[i]]
            mTime[ind[i]] += mTest[i][ind[i]]
#         print(mTime)
#         print(tTime)
    score = max(mTime) / sum(mTime)
    if(isnan(score)): score = 1
    return 1 - score

b = [1, 1, 1, 1, 0, 1, 1, 2]
fitness(b)
# a = [fitness(i) for i in gen]
# print(gen)
# print(a)

In [159]:
# Funcion general
# nIndGen: numero de individuos por generacion
# nGen: numero de generaciones que realizara el algoritmo
# pMut: probabilidad de mutacion
from time import time
start_time = time()
def genetico(nIndGen,nGen,pMut, function):
    generacion = primeraGen(nIndGen)
    while nGen > 0:
        generacion.sort(reverse = True, key = fitness)
        generacion = descarte(generacion)
        children = []
        while len(children) + len(generacion) < nIndGen:
            parent1, parent2 = function(generacion)
            child1, child2 = cruce(parent1,parent2)
            child1 = mutacion(child1, pMut)
            child2 = mutacion(child2, pMut)
            children.append(child1)
            children.append(child2)
        generacion = generacion + children
        nGen = nGen - 1
    generacion.sort(reverse = True, key = fitness)
    print(generacion,"\n")
    return [ fitness(i) for i in generacion]
#     return generacion
print(genetico(50, 4, 0.1, roulette))
print("--- %s seconds ---" % (time() - start_time))

[[0, 1, 2, 1, 0, 2, 2, 1], [0, 1, 2, 1, 0, 2, 2, 1], [0, 1, 2, 1, 0, 2, 2, 1], [0, 1, 2, 1, 0, 1, 2, 1], [0, 1, 2, 1, 0, 1, 2, 2], [0, 1, 2, 1, 0, 1, 2, 2], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1, 0, 2, 0, 1], [0, 1, 2, 1